In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn import metrics

%matplotlib inline

# Read files into dataframe
train_df = pd.read_csv("./data/health-diagnostics-train.csv", na_values=["#NULL!"])
test_df = pd.read_csv("./data/health-diagnostics-test.csv", na_values=["#NULL!"])
# test_sample = pd.read_csv("./data/health-diagnostics-test-sample-solution.csv")

feature_cols = ['income', 'maternal', 'fam-history', 'mat-illness-past', 'suppl', 'mat-illness', 'meds', 'env', 'lifestyle']

# Replace missing values with mode and change objects to integers
def replace_nulls(df):
    for f in feature_cols:
        df[f] = df[f].astype(float)
        df[f].fillna(df[f].mode()[0], inplace=True)
        df[f] = df[f].astype(int)

replace_nulls(test_df)
replace_nulls(train_df)

**Domain:**
- income - an annual per capita income of a patient
- maternal - maternal delivery age
- fam-history - a family history
- mat-illness-past - a previous maternal illness history
- suppl - nutrition and folic acid supplementation
- mat-illness - a maternal illness
- meds - medication use
- env - an environmental exposures of risk factors
- lifestyle - an unhealthy lifestyle
- target - a congenital disorder

In [ ]:
# train_df.shape
# train_df.dtypes
# train_df.isnull().sum()
# train_df.describe()

In [ ]:
# for f in feature_cols:
# #     display(f, train_df[f].value_counts())
#     train_df[f].sort_values().value_counts(sort=False).plot(kind="bar")
#     plt.show();

- income
    - even distribution
- maternal
    - large mode of 0
- fam-history
    - large mode of 0
- mat-illness-past
    - large mode of 0
- suppl
    - left skewed
- mat-illness
- meds
- env
    - has a distribution
- lifestyle
    - has a distribution, may need to create dummies

In [ ]:
train_df['target'].value_counts()

- shows binary classification (0 or 1), so requires logistic regression
- shows there is a class imbalance, very few 1's

In [ ]:
# Find train to test ratio
train_df.shape[0] / (train_df.shape[0]+test_df.shape[0])

# Find percentage of 1's
round(train_df['target'].sum() / train_df['target'].count() * 100, 2)

- shows there is a 90 : 10 split between train : test
- therefore, probably very few true predictions
- there is there is 0.18% chance of target 1 for every row

In [ ]:
sns.heatmap(train_df.corr())

In [ ]:
target_mask = train_df['target'] == 1

feature_cols = ['income', 'maternal', 'fam-history', 'mat-illness-past', 'suppl', 'mat-illness', 'meds', 'env', 'lifestyle']

# Find percentage of 1 occuring out of total for each column each category
for f in feature_cols:
    display(f, train_df[target_mask][f].value_counts(sort=False), train_df[f].value_counts(sort=False),
            train_df[target_mask][f].value_counts(sort=False) / train_df[f].value_counts(sort=False) * 100)

In [ ]:
# def train_test_rmse(df, feature_cols):
#     X = df[feature_cols]
#     y = df.target
    
#     X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
    
#     linreg = LinearRegression()
#     linreg.fit(X_train, y_train)
    
#     y_pred = linreg.predict(X_test)
#     return np.sqrt(metrics.mean_squared_error(y_test, y_pred))

# print(train_test_rmse(train_df, feature_cols, ['target']))

**Logistic Regression**

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix

feature_cols = ['income', 'maternal', 'fam-history', 'mat-illness-past', 'suppl', 'mat-illness', 'meds', 'env', 'lifestyle']

X = train_df[feature_cols]
y = train_df.target

# Split X and y into training and testing sets.
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

logreg = LogisticRegression()
logreg.fit(X_train, y_train)

y_pred = logreg.predict(X_test)

# Displaying confusion matrix
display(pd.crosstab(y_test, y_pred, rownames=['True'], colnames=['Predicted'], margins=True))
recall_metric = round(100*cnf_matrix_tra[1,1]/(cnf_matrix_tra[1,0]+cnf_matrix_tra[1,1]),0)
print("Recall metric in the train dataset: {0}%".format(recall_metric))

# Finding the f1 score
display(metrics.f1_score(y_test, y_pred, average="binary"))

# Predicting on test data
test_df['target'] = logreg.predict(test_df[feature_cols])

# Saving to csv for submission
test_df['target'].to_csv("./data/logreg.csv", index_label="index", header="target")

- 0.58 AUC

**Applying feature engineering**

In [ ]:
from sklearn.linear_model import LogisticRegression

# Feature engineering
train_df['low_income'] = train_df['income'] == 1  # higher chance of target
train_df['low_suppl'] = train_df['suppl'] < 2  # higher chance of target
train_df['mat_ill_cat'] = train_df['mat-illness'] == 2   # higher chance of target
lx_dummies = pd.get_dummies(train_df['lifestyle'], prefix="lx")
lx_dummies.drop(lx_dummies.columns[0], axis=1, inplace=True)
train_df = pd.concat([train_df, lx_dummies], axis=1)

feature_cols = ['low_income', 'maternal', 'fam-history', 'mat-illness-past', 'low_suppl', 'mat_ill_cat', 
                'meds', 'lx_1', 'lx_2', 'lx_3', 'lx_4', 'lx_5', 'lx_6', 'lx_7', 'lx_8']

X = train_df[feature_cols]
y = train_df.target

# Split X and y into training and testing sets.
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

logreg = LogisticRegression()
logreg.fit(X_train, y_train)

y_pred = logreg.predict(X_test)

# Displaying confusion matrix
display(pd.crosstab(y_test, y_pred, rownames=['True'], colnames=['Predicted'], margins=True))
recall_metric = round(100*cnf_matrix_tra[1,1]/(cnf_matrix_tra[1,0]+cnf_matrix_tra[1,1]),0)
print("Recall metric in the train dataset: {0}%".format(recall_metric))

metrics.f1_score(y_test, y_pred, average="binary")

- no improvement

binary classification with unbalanced data

https://bmcbioinformatics.biomedcentral.com/articles/10.1186/1471-2105-14-106
https://www.marcoaltini.com/blog/dealing-with-imbalanced-data-undersampling-oversampling-and-proper-cross-validation

**K-nearest neighbour**

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics

feature_cols = ['income', 'maternal', 'fam-history', 'mat-illness-past', 'suppl', 'mat-illness', 'meds', 'env', 'lifestyle']

X = train_df[feature_cols]
y = train_df.target

# Split X and y into training and testing sets.
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)

y_pred = knn.predict(X_test)

# Displaying confusion matrix
display(pd.crosstab(y_test, y_pred, rownames=['True'], colnames=['Predicted'], margins=True))
# recall_metric = round(100*cnf_matrix_tra[1,1]/(cnf_matrix_tra[1,0]+cnf_matrix_tra[1,1]),0)
# print("Recall metric in the train dataset: {0}%".format(recall_metric))

metrics.f1_score(y_test, y_pred, average="binary")

# scores = []

# for k in range(1,10):
#     knn = KNeighborsClassifier(n_neighbors=k)
#     knn.fit(X_train, y_train)
#     y_pred = knn.predict(X_test)
#     scores.append([k, metrics.f1_score(y_test, y_pred, average="binary")])

# print(scores)

**Syntheic Minority Over-sampling Technique**

In [ ]:
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, precision_recall_curve, auc, roc_auc_score, roc_curve, recall_score, classification_report

feature_cols = ['income', 'maternal', 'fam-history', 'mat-illness-past', 'suppl', 'mat-illness', 'meds', 'env', 'lifestyle']

X = train_df[feature_cols]
y = train_df.target

# Split X and y into training and testing sets.
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

print("Number transactions X_train dataset: ", X_train.shape)
print("Number transactions y_train dataset: ", y_train.shape)
print("Number transactions X_test dataset: ", X_test.shape)
print("Number transactions y_test dataset: ", y_test.shape)
print("Before oversampling, counts of label '1': {}".format(sum(y_train==1)))
print("Before oversampling, counts of label '0': {} \n".format(sum(y_train==0)))

sm = SMOTE(random_state=42)
X_train_res, y_train_res = sm.fit_sample(X_train, y_train.ravel())

print('After oversampling, the shape of train_X: {}'.format(X_train_res.shape))
print('After oversampling, the shape of train_y: {} \n'.format(y_train_res.shape))
print("After oversampling, counts of label '1': {}".format(sum(y_train_res==1)))
print("After oversampling, counts of label '0': {}".format(sum(y_train_res==0)))

parameters = {
    'C': np.linspace(1, 10, 10)
             }
lr = LogisticRegression()
clf = GridSearchCV(lr, parameters, cv=5, verbose=5, n_jobs=3)
clf.fit(X_train_res, y_train_res.ravel())

clf.best_params_

In [ ]:
lr1 = LogisticRegression(C=2, penalty='l1', verbose=5)
lr1.fit(X_train_res, y_train_res.ravel())

In [ ]:
import itertools

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=0)
    plt.yticks(tick_marks, classes)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

y_train_pre = lr1.predict(X_train)

cnf_matrix_tra = confusion_matrix(y_train, y_train_pre)

print("Recall metric in the train dataset: {}%".format(100*cnf_matrix_tra[1,1]/(cnf_matrix_tra[1,0]+cnf_matrix_tra[1,1])))

class_names = [0,1]
plt.figure()
plot_confusion_matrix(cnf_matrix_tra , classes=class_names, title='Confusion matrix')
plt.show()

In [ ]:
y_pre = lr1.predict(X_test)

cnf_matrix = confusion_matrix(y_test, y_pre)

print("Recall metric in the testing dataset: {}%".format(100*cnf_matrix[1,1]/(cnf_matrix[1,0]+cnf_matrix[1,1])))
#print("Precision metric in the testing dataset: {}%".format(100*cnf_matrix[0,0]/(cnf_matrix[0,0]+cnf_matrix[1,0])))
# Plot non-normalized confusion matrix
class_names = [0,1]
plt.figure()
plot_confusion_matrix(cnf_matrix , classes=class_names, title='Confusion matrix')
plt.show()

In [ ]:
tmp = lr1.fit(X_train_res, y_train_res.ravel())

In [ ]:
y_pred_sample_score = tmp.decision_function(X_test)

fpr, tpr, thresholds = roc_curve(y_test, y_pred_sample_score)

roc_auc = auc(fpr,tpr)

# Plot ROC
plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b',label='AUC = %0.3f'% roc_auc)
plt.legend(loc='lower right')
plt.plot([0,1],[0,1],'r--')
plt.xlim([-0.1,1.0])
plt.ylim([-0.1,1.01])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

In [ ]:
lr1.fit(X_train_res, y_train_res.ravel())

# Predicting on test data
test_df['target'] = lr1.predict(test_df[feature_cols])

# Saving to csv for submission
test_df['target'].to_csv("./data/logreg_smote.csv", index_label="index", header="target")

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics

feature_cols = ['income', 'maternal', 'fam-history', 'mat-illness-past', 'suppl', 'mat-illness', 'meds', 'env', 'lifestyle']

X = train_df[feature_cols]
y = train_df.target

# Split X and y into training and testing sets.
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

print("Number transactions X_train dataset: ", X_train.shape)
print("Number transactions y_train dataset: ", y_train.shape)
print("Number transactions X_test dataset: ", X_test.shape)
print("Number transactions y_test dataset: ", y_test.shape)
print("Before oversampling, counts of label '1': {}".format(sum(y_train==1)))
print("Before oversampling, counts of label '0': {} \n".format(sum(y_train==0)))

sm = SMOTE(random_state=42)
X_train_res, y_train_res = sm.fit_sample(X_train, y_train.ravel())

print('After oversampling, the shape of train_X: {}'.format(X_train_res.shape))
print('After oversampling, the shape of train_y: {} \n'.format(y_train_res.shape))
print("After oversampling, counts of label '1': {}".format(sum(y_train_res==1)))
print("After oversampling, counts of label '0': {}".format(sum(y_train_res==0)))

parameters = {
    'C': np.linspace(1, 10, 10)
             }
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)

clf = GridSearchCV(lr, parameters, cv=5, verbose=5, n_jobs=3)
clf.fit(X_train_res, y_train_res.ravel())

clf.best_params_